In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_json("datos.json", orient='records')
df

,_id,codigo,tipo_propiedad,tipo_operacion,precio_venta,area,habitaciones,banos,administracion,parqueaderos,...,vigilancia,coords_modified,localidad,barrio,estacion_tm_cercana,distancia_estacion_tm_m,is_cerca_estacion_tm,parque_cercano,distancia_parque_m,is_cerca_parque
0,{'$oid': '66d86c7eceda690e85508760'},4133491,APARTAMENTO,VENTA,339000000,76.00,3,2,300000,1,...,0,False,SUBA,URB. PICADELLY,Toberin - Foundever,1142.45,0,PARQUE VECINAL DESARROLLO CANTALEJO URBANIZACI...,426.09,1
1,{'$oid': '66d86c7eceda690e85508761'},3889852,APARTAMENTO,VENTA,223000000,63.00,3,2,{'$numberDouble': 'NaN'},0,...,0,False,USAQUEN,BARRANCAS PERLAS DEL NORTE,Mazurén,2384.89,0,PARQUE ZONAL ALTA BLANCA,472.47,1
2,{'$oid': '66d86c7eceda690e85508762'},4386770,APARTAMENTO,VENTA,440898168,54.00,3,2,305000,0,...,0,False,LOS MARTIRES,SAN VICTORINO,De La Sabana,232.22,1,PARQUE METROPOLITANO TERCER MILENIO,961.29,0
3,{'$oid': '66d86c7eceda690e85508763'},4210060,APARTAMENTO,VENTA,158000000,43.00,2,2,106600,0,...,0,False,SUBA,LA ESTRELLA II,Portal Suba,2275.08,0,PARQUE ZONAL LA GAITANA,539.98,0
4,{'$oid': '66d86c7eceda690e85508764'},4063762,APARTAMENTO,VENTA,222800000,48.00,3,2,151000,0,...,0,False,USAQUEN,HORIZONTES USAQUEN,Terminal,2099.16,0,PARQUE VECINAL DESARROLLO VERBENAL I,1661.14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43008,{'$oid': '66d86ebfceda690e85512f60'},4323513,APARTAMENTO,VENTA,190000000,49.00,3,2,204624,0,...,0,False,SUBA,URB. OVIEDO,Portal Suba,1712.71,0,PARQUE ZONAL CAMPO VERDE,198.03,1
43009,{'$oid': '66d86ebfceda690e85512f61'},4345085,APARTAMENTO,VENTA,322000000,66.00,3,2,277400,1,...,0,False,SUBA,LOS TEJARES DE SUBA (PREDIO),La Campiña,120.16,1,PARQUE ZONAL BELLAVISTA DINDALITO,631.25,0
43010,{'$oid': '66d86ebfceda690e85512f62'},MC5206960,APARTAMENTO,VENTA,330000000,90.00,3,2,112000,0,...,0,False,TEUSAQUILLO,LAS AMERICAS,AV. ElDorado,33.21,1,PARQUE METROPOLITANO EL RENACIMIENTO - PARQUE ...,1211.62,0
43011,{'$oid': '66d86ebfceda690e85512f63'},MC5203688,APARTAMENTO,VENTA,1280000000,157.00,3,3,1050000,3,...,0,False,USAQUEN,S.C. SAN NORTE,Calle 127,3005.88,0,PARQUE METROPOLITANO EL COUNTRY,1269.56,0


In [5]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43013 entries, 0 to 43012
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      43013 non-null  object 
 1   codigo                   43013 non-null  object 
 2   tipo_propiedad           43013 non-null  object 
 3   tipo_operacion           43013 non-null  object 
 4   precio_venta             43013 non-null  object 
 5   area                     43013 non-null  float64
 6   habitaciones             43013 non-null  object 
 7   banos                    43013 non-null  object 
 8   administracion           43013 non-null  object 
 9   parqueaderos             43013 non-null  object 
 10  sector                   43013 non-null  object 
 11  estrato                  43013 non-null  object 
 12  antiguedad               43013 non-null  object 
 13  latitud                  43013 non-null  float64
 14  longitud              

,area,latitud,longitud,distancia_estacion_tm_m,is_cerca_estacion_tm,distancia_parque_m,is_cerca_parque
count,4.301300e+04,43013.000000,43013.000000,43013.000000,43013.000000,43013.000000,43013.000000
mean,1.800341e+02,4.686099,-74.062808,1270.515946,0.209495,808.884766,0.275033
std,9.163002e+03,0.038297,0.033544,841.814012,0.406953,447.796211,0.446536
min,0.000000e+00,4.468629,-74.213645,4.940000,0.000000,0.220000,0.000000
25%,6.300000e+01,4.663322,-74.068665,578.380000,0.000000,470.140000,0.000000
50%,1.000000e+02,4.688729,-74.052000,1175.520000,0.000000,750.140000,0.000000
75%,1.740000e+02,4.711980,-74.043491,1707.920000,0.000000,1087.730000,1.000000
max,1.900000e+06,4.820822,-74.014000,7095.660000,1.000000,6276.510000,1.000000
